In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
%%time

import malaya
malaya

CPU times: user 2.85 s, sys: 2.61 s, total: 5.46 s
Wall time: 2.41 s


/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


<module 'malaya' from '/home/ubuntu/dev/malaya/malaya/__init__.py'>

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
sastrawi = malaya.stem.sastrawi()

In [5]:
# !wget https://github.com/huseinzol05/malay-dataset/blob/master/normalization/stemmer/stem.zip?raw=true
# !wget https://f000.backblazeb2.com/file/malay-dataset/wiki-stem.json
# !unzip stem.zip?raw=true

In [6]:
with open('/home/ubuntu/malaya/stemmer-data-v5.txt') as fopen:
    stemming = fopen.read().split('\n')
stemming[:5]

['spider\tspider',
 'transeksual\ttranseksual',
 'khusus\tkhusus',
 'kriminologi\tkriminologi',
 'negeripingat\tnegeripingat']

In [7]:
left, right = [], []

for s in stemming:
    l, r = s.split('\t')
    left.append(l)
    right.append(r)
    
len(left)

41528

In [8]:
import json

with open('/home/ubuntu/malaya/news-stem.json') as fopen:
    news = json.load(fopen)
    
for s in news:
    left.append(s[0])
    right.append(s[1])

In [9]:
with open('/home/ubuntu/malaya/wiki-stem.json') as fopen:
    wiki = json.load(fopen)
    
for s in wiki:
    left.append(s[0])
    right.append(s[1])

In [10]:
len(left), len(right)

(1832338, 1832338)

In [11]:
left[1], right[0]

('transeksual', 'spider')

In [12]:
sastrawi.stem(f'{left[1]}lah')

'transeksual'

In [13]:
stem = sastrawi.stem('sakitnya')
stem

'sakit'

In [14]:
hujung = {
    'kan': ['kann', 'kkan', 'kaann', 'kn', 'kkn'],
    'lah': ['lh', 'lhh', 'lhaa', 'lha', 'laah', 'laahh', 'ler', 'la'],
    'nya': ['nye', 'ny', 'nyee', 'nyye', 'nyo', 'nyoo'],
}

In [15]:
import re
import random

vowels = 'aeiou'
consonants = 'bcdfghjklmnpqrstvwxyz'

In [16]:
def augment_hubung(word, p_remove_vowels = 0.5, p_duplicate = 0.7, min_duplicate = 2, max_duplicate = 3):
    word = ['' if c in vowels and random.random() > p_remove_vowels else c for c in word]
    word = list(filter(None, word))
    for i in range(len(word)):
        w = word[i]
        if random.random() > p_duplicate:
            duplicate = random.randint(min_duplicate, max_duplicate)
            word[i] = w * duplicate
    return ''.join(word)

In [17]:
malaya.augmentation.vowel_alternate('sakit')

'sakit'

In [18]:
augment_hubung('ter')

'teerrr'

In [19]:
string = 'tersakitnya'
stemmed = sastrawi.stem(string)
index = [(m.start(0), m.end(0)) for m in re.finditer(stemmed, string, flags=re.IGNORECASE)][0]
permulaan = string[:index[0]]
hujungan = string[index[1]:]
permulaan, hujungan

('ter', 'nya')

In [20]:
awalan = ['pe', 'pem', 'pen', 'peng', 'pel', 'ke', 'juru', 'pra', 'dwi',
         'me', 'memper', 'ber', 'ter', 'di', 'diper', 'ter', 'te', 'se',
         'mem', 'diper']

akhiran = ['lah', 'an', 'kan', 'nya', 'wan', 'wati']

In [21]:
random.choice(hujung.get(hujungan, [hujungan]))

'nyo'

In [22]:
sastrawi.stem('memperhambakan')

'hamba'

In [23]:
set_left = set(left)

In [24]:
from tqdm import tqdm

In [25]:
new_left, new_right = [], []
for w in tqdm(malaya.dictionary.bahasa.dbp.words | malaya.dictionary.bahasa.cambridge_words.words | malaya.dictionary.bahasa.kamus_dewan.words | malaya.dictionary.bahasa.words.words):
    if w in set_left:
        continue
    s = sastrawi.stem(w)
    new_left.append(w)
    new_right.append(s)
    set_left.add(w)
    
left.extend(new_left)
right.extend(new_right)

100%|██████████████████████████████████████████████████████████████████████████████████| 63838/63838 [00:03<00:00, 18127.31it/s]


In [26]:
len(left), len(right)

(1855109, 1855109)

In [27]:
import numpy as np

In [28]:
np.unique(list('nn')).shape[0]

1

In [29]:
def augment(word, p_augment_word = 0.7):
    awal = random.choice(awalan)
    akhir = random.choice(akhiran)
    joined = awal + word + akhir
    stemmed = sastrawi.stem(joined)
    if stemmed == joined:
        return word, word
    
    index = [(m.start(0), m.end(0)) for m in re.finditer(stemmed, joined, flags=re.IGNORECASE)][0]
    permulaan = joined[:index[0]]
    hujungan = joined[index[1]:]
    
    if not len(permulaan) and not len(hujungan):
        return False, left, right
    
    print(joined, stemmed, permulaan, hujungan)
    
    augmented_permulaan = augment_hubung(permulaan)
    if len(augmented_permulaan) > 1:
        permulaan = augmented_permulaan
    
    augmented_hujungan = augment_hubung(hujungan)
    if len(augmented_hujungan) > 1:
        hujungan = augmented_hujungan
        
    if random.random() > p_augment_word:
        word = malaya.augmentation.vowel_alternate(word)
    
    return True, permulaan + word + hujungan, word

def augment_hujung(word, p_augment_word = 0.7):
    akhir = random.choice(akhiran)
    joined = word + akhir
    stemmed = sastrawi.stem(joined)
    if stemmed == joined:
        return False, word, word
    
    index = [(m.start(0), m.end(0)) for m in re.finditer(stemmed, joined, flags=re.IGNORECASE)][0]
    hujungan = joined[index[1]:]
    
    if not len(hujungan):
        return False, word, word
    
    if hujungan in hujung:
        hujungan = random.choice(hujung.get(hujungan, [hujungan]))
    else:
        augmented_hujungan = augment_hubung(hujungan)
        if len(augmented_hujungan) > 1:
            hujungan = augmented_hujungan
            
    if np.unique(list(hujungan)).shape[0] == 1:
        return False, word, word
        
    if random.random() > p_augment_word:
        word = malaya.augmentation.vowel_alternate(word)
    
    return True, word + hujungan, word

def augment_left_right(left, right, augment_hubungan = True, p_augment_word = 0.7):
    index = [(m.start(0), m.end(0)) for m in re.finditer(right, left, flags=re.IGNORECASE)]
    permulaan = left[:index[0][0]]
    hujungan = left[index[-1][1]:]

    not_right = left[index[0][0]:index[-1][1]]
    # print(permulaan, hujungan, not_right)
    
    if not len(permulaan) and not len(hujungan):
        return False, left, right
    
    if augment_hubungan:
        augmented_permulaan = augment_hubung(permulaan)
        if len(augmented_permulaan) > 1:
            permulaan = augmented_permulaan
            
    if np.unique(list(permulaan)).shape[0] == 1:
        return False, left, right
    
    if augment_hubungan:
        augmented_hujungan = augment_hubung(hujungan)
        if len(augmented_hujungan) > 1:
            hujungan = augmented_hujungan
            
    if np.unique(list(hujungan)).shape[0] == 1:
        return False, left, right
    
    if random.random() > p_augment_word and not_right == right:
        right = malaya.augmentation.vowel_alternate(right)
        selected = right
    else:
        selected = not_right
    
    joined = permulaan + selected + hujungan
    joined = re.sub(r'^-+|-+$', '-', joined).strip()
    
    
    if joined == left:
        result = False
    else:
        result = True
    
    return result, joined, right

In [30]:
left[:10]

['spider',
 'transeksual',
 'khusus',
 'kriminologi',
 'negeripingat',
 'melingkungi',
 'terlajak',
 'wooi',
 'penguin',
 'agung']

In [31]:
for i in range(50):
    print(augment_hujung('agung'))

(True, 'agungnyo', 'agung')
(True, 'agungan', 'agung')
(False, 'agung', 'agung')
(True, 'agungaannn', 'agung')
(True, 'agungnyoo', 'agung')
(True, 'agngkkn', 'agng')
(False, 'agung', 'agung')
(True, 'agungny', 'agung')
(True, 'agnglhh', 'agng')
(False, 'agung', 'agung')
(False, 'agung', 'agung')
(True, 'agungan', 'agung')
(True, 'agungkkn', 'agung')
(False, 'agung', 'agung')
(False, 'agung', 'agung')
(True, 'agnglaah', 'agng')
(False, 'agung', 'agung')
(False, 'agung', 'agung')
(True, 'agungnyee', 'agung')
(True, 'agungkaann', 'agung')
(False, 'agung', 'agung')
(False, 'agung', 'agung')
(True, 'agungaann', 'agung')
(False, 'agung', 'agung')
(True, 'agunglaahh', 'agung')
(True, 'agungkkn', 'agung')
(False, 'agung', 'agung')
(True, 'agunglhh', 'agung')
(False, 'agung', 'agung')
(False, 'agung', 'agung')
(True, 'agungnye', 'agung')
(False, 'agung', 'agung')
(True, 'agungkann', 'agung')
(True, 'agungnyo', 'agung')
(False, 'agung', 'agung')
(True, 'agungan', 'agung')
(False, 'agung', 'agung

In [32]:
for i in range(50):
    print(augment_left_right('menganggapnya', 'anggap'))

(True, 'mnganggapnya', 'anggap')
(True, 'mmeeenganggapnnnyaaa', 'anggap')
(True, 'mnnngganggapnnyaaa', 'anggap')
(True, 'mnganggapnyy', 'anggap')
(True, 'mmmennnganggapny', 'anggap')
(True, 'menganggapny', 'anggap')
(True, 'mnnnganggapnya', 'anggap')
(True, 'mmnganggpnny', 'anggp')
(True, 'mmnnnganggapnyaa', 'anggap')
(True, 'mmmnganggapnyy', 'anggap')
(True, 'menganggapnnnyya', 'anggap')
(True, 'menganggapnnnya', 'anggap')
(True, 'mnnnganggapnyyyaaa', 'anggap')
(True, 'mmenggganggapnya', 'anggap')
(True, 'meenngganggapnnyyya', 'anggap')
(True, 'mnganggapnya', 'anggap')
(True, 'mnganggapnyaa', 'anggap')
(True, 'mnnngganggapny', 'anggap')
(True, 'mmenganggapnya', 'anggap')
(True, 'mmmnggganggpnyyya', 'anggp')
(True, 'mnggganggapnyyya', 'anggap')
(True, 'mmmenganggapnya', 'anggap')
(True, 'meeenganggapnyyya', 'anggap')
(True, 'meeengganggapnyyy', 'anggap')
(True, 'meennggganggapny', 'anggap')
(True, 'mmnganggapny', 'anggap')
(True, 'mmmnganggapnny', 'anggap')
(True, 'menganggapnyyya', 'a

In [33]:
# def cleaning(word):
#     c = re.sub(r'^-+|-+$', '-', word.lower()).strip().replace('"', ' ').replace('\'', ' ')
#     c = re.sub(r'[ ]+', ' ', c).strip()
#     if len(c) and c[-1] in '-':
#         c = c[:-1]
#         c = re.sub(r'[ ]+', ' ', c).strip()
#     if len(c) and c[0] in '-':
#         c = c[1:]
#         c = re.sub(r'[ ]+', ' ', c).strip()
#     return c

# left_, right_ = [], []
# for i in range(len(left)):
#     c_l = cleaning(left[i])
#     if len(c_l) > 1 and c_l[-1] not in '-' and c_l[0] not in '-' and ' ' not in c_l and c_l.count('-') <= 1 and all([len(a) > 1 for a in c_l.split('-')]):
#         left_.append(c_l)
#     c_r = cleaning(right[i])
#     if len(c_r) > 1 and c_r[-1] not in '-' and c_r[0] not in '-' and ' ' not in c_r and c_r.count('-') <= 1 and all([len(a) > 1 for a in c_l.split('-')]):
#         right_.append(c_r)

In [34]:
# sorted_ = sorted(list(set(left_ + right_)))

In [35]:
# with open('unique-words.json', 'w') as fopen:
#     json.dump(sorted_, fopen)

In [36]:
def cleaning(word):
    c = re.sub(r'^-+|-+$', '-', word).strip().replace('"', ' ').replace('\'', ' ')
    c = re.sub(r'[ ]+', ' ', c).strip()
    if len(c) and c[-1] in '-':
        c = c[:-1]
        c = re.sub(r'[ ]+', ' ', c).strip()
    if len(c) and c[0] in '-':
        c = c[1:]
        c = re.sub(r'[ ]+', ' ', c).strip()
    return c

In [37]:
for i in tqdm(range(len(left))):
    left[i] = cleaning(left[i])

100%|█████████████████████████████████████████████████████████████████████████████| 1855109/1855109 [00:02<00:00, 658773.84it/s]


In [38]:
combined = []
for i in tqdm(range(len(left))):
    if ' ' in left[i]:
        continue
    if len(left[i]) < 2:
        continue
    if len(right[i]) < 2:
        continue
    if left[i].count('-') > 1:
        continue
    if right[i][0] in '-.':
        continue
    if right[i][-1] in '-.':
        continue
    combined.append(f'{left[i]}<<>>{right[i]}')

100%|████████████████████████████████████████████████████████████████████████████| 1855109/1855109 [00:01<00:00, 1700055.63it/s]


In [39]:
filtered_left, filtered_right = [], []
for c in set(combined):
    if '<<>>' in c:
        l, r = c.split('<<>>')
        filtered_left.append(l)
        filtered_right.append(r)

In [40]:
len(filtered_left), len(combined), len(left)

(1228060, 1796170, 1855109)

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(filtered_left, filtered_right, test_size = 0.1)

In [ ]:
with open('train_stem.json', 'w') as fopen:
    json.dump({'X': train_X, 'Y': train_Y}, fopen)

In [ ]:
with open('test_stem.json', 'w') as fopen:
    json.dump({'X': test_X, 'Y': test_Y}, fopen)

In [41]:
not_same = []
for i in tqdm(range(len(left))):
    left_ = left[i].lower()
    left_ = re.sub(r'^-+|-+$', '-', left_).strip().replace('"', ' ').replace('\'', ' ')
    left_ = re.sub(r'[ ]+', ' ', left_).strip()
    right_ = right[i].lower()
    
    if left_ != right_ and left[i].lower() != right[i].lower():
        left_ = cleaning(left[i])
        not_same.append((left_, right[i]))
        
len(not_same), len(not_same) / len(left)

100%|█████████████████████████████████████████████████████████████████████████████| 1855109/1855109 [00:03<00:00, 586507.14it/s]


(198320, 0.10690476947715741)

In [42]:
not_same[-1000:]

[('cacingan', 'cacing'),
 ('pertaruh', 'taruh'),
 ('bering-bering', 'ring'),
 ('mengukirnya', 'ukir'),
 ('menciut-ciut', 'ciut'),
 ('mentakwilkan', 'takwil'),
 ('dikepal-kepal', 'kepal'),
 ('menggaduk', 'gaduk'),
 ('berkeritik-keritik', 'keritik'),
 ('bersudah', 'sudah'),
 ('terpaling-paling', 'paling'),
 ('berbedan', 'bedan'),
 ('menengil', 'tengil'),
 ('didongengkan', 'dongeng'),
 ('guntang-guntang', 'guntang'),
 ('berpiak', 'piak'),
 ('menggeramkan', 'geram'),
 ('bercelur', 'celur'),
 ('penjajaan', 'jaja'),
 ('tersekang', 'sekang'),
 ('mendangkalkan', 'dangkal'),
 ('berperdua', 'dua'),
 ('meligih', 'ligih'),
 ('nipiskan', 'nipis'),
 ('bertongkang', 'tongkang'),
 ('kegelian', 'geli'),
 ('berhitung-hitung', 'hitung'),
 ('bercerai-ceraian', 'cerai'),
 ('bertetar', 'tetar'),
 ('sampai-menyampai', 'sampai'),
 ('menyendat', 'sendat'),
 ('menyenak', 'senak'),
 ('lingkah', 'ling'),
 ('melesang', 'lesang'),
 ('kehangitan', 'hangit'),
 ('membuayai', 'buaya'),
 ('setarafkan', 'taraf'),
 ('meng

In [43]:
for i in range(10):
    print(i, augment_left_right('mengenjak-enjak', 'enjak'))

0 (True, 'mmmnggenjak-enjak', 'enjak')
1 (True, 'mmmeengenjak-enjak', 'enjak')
2 (True, 'mennngenjak-enjak', 'enjak')
3 (True, 'mngenjak-enjak', 'enjak')
4 (True, 'mmmngenjak-enjak', 'enjak')
5 (True, 'mnngenjak-enjak', 'enjak')
6 (True, 'menngenjak-enjak', 'enjak')
7 (True, 'mngenjak-enjak', 'enjak')
8 (False, 'mengenjak-enjak', 'enjak')
9 (True, 'menggenjak-enjak', 'enjak')


In [44]:
'<<>>'.join(augment_left_right('mengenjak-enjak', 'enjak')[1:])

'mmmeeengenjak-enjak<<>>enjak'

In [45]:
for i in range(50):
    print(i, '<<>>'.join(augment_left_right('membidai', 'bidai', augment_hubungan = False, 
                                           p_augment_word = 0.85)[1:]))

0 membidai<<>>bidai
1 membidai<<>>bidai
2 membidai<<>>bidai
3 membdai<<>>bdai
4 membidai<<>>bidai
5 membidai<<>>bidai
6 membidai<<>>bidai
7 membidai<<>>bidai
8 membidai<<>>bidai
9 membidai<<>>bidai
10 membidai<<>>bidai
11 membidai<<>>bidai
12 membidai<<>>bidai
13 membidai<<>>bidai
14 membidai<<>>bidai
15 membidai<<>>bidai
16 membidai<<>>bidai
17 membidai<<>>bidai
18 membdai<<>>bdai
19 membidai<<>>bidai
20 membidai<<>>bidai
21 membidai<<>>bidai
22 membidai<<>>bidai
23 membidai<<>>bidai
24 membidai<<>>bidai
25 membidai<<>>bidai
26 membidai<<>>bidai
27 membidai<<>>bidai
28 membidai<<>>bidai
29 membidai<<>>bidai
30 membidai<<>>bidai
31 membidai<<>>bidai
32 membidai<<>>bidai
33 membidai<<>>bidai
34 membidai<<>>bidai
35 membidai<<>>bidai
36 membidai<<>>bidai
37 membidai<<>>bidai
38 membidai<<>>bidai
39 membidai<<>>bidai
40 membidai<<>>bidai
41 membdai<<>>bdai
42 membidai<<>>bidai
43 membidai<<>>bidai
44 membidai<<>>bidai
45 membidai<<>>bidai
46 membidai<<>>bidai
47 membidai<<>>bidai
48 membi

In [46]:
for n in tqdm(not_same):
    for _ in range(3):
        try:
            
            l = augment_left_right(n[0], n[1], augment_hubungan = False, p_augment_word = 0.85)
            if l[0]:
                combined.append('<<>>'.join(l[1:]))
        except:
            pass
        
    for _ in range(7):
        try:
            l = augment_left_right(*n)
            if l[0]:
                combined.append('<<>>'.join(l[1:]))
        except:
            pass

100%|█████████████████████████████████████████████████████████████████████████████████| 198320/198320 [00:35<00:00, 5526.65it/s]


In [47]:
len(filtered_left)

1228060

In [48]:
augment_hujung('agung')

(False, 'agung', 'agung')

In [51]:
sampled = []
for w in tqdm(malaya.dictionary.bahasa.dbp.words | malaya.dictionary.bahasa.cambridge_words.words | malaya.dictionary.bahasa.kamus_dewan.words | malaya.dictionary.bahasa.words.words):
    s = sastrawi.stem(w)
    sampled.append(s)
    
sampled[:10]

100%|██████████████████████████████████████████████████████████████████████████████████| 63838/63838 [00:05<00:00, 11356.69it/s]


['jauhar',
 'lecut',
 'perkhatanan',
 'pegas',
 'seranah',
 'terka',
 'maki',
 'bersempudal',
 'cacat',
 'gadai']

In [55]:
new_sampled = set(sampled + [i[1] for i in not_same] + random.sample())
len(new_sampled)

41817

In [58]:
for r in tqdm(new_sampled):
    a = []
    try:
        for _ in range(20):
            l = augment_hujung(r)
            if l[0]:
                a.append('<<>>'.join(l[1:]))
    except:
        pass
    a = list(set(a))
    combined.extend(a)

100%|████████████████████████████████████████████████████████████████████████████████████| 41817/41817 [01:24<00:00, 494.48it/s]


In [60]:
len(combined)

2773954

In [61]:
combined = set(combined)

In [62]:
len(set(combined))

2035197

In [64]:
filtered_left, filtered_right = [], []
for c in combined:
    if '<<>>' in c:
        l, r = c.split('<<>>')
        filtered_left.append(l)
        filtered_right.append(r)

In [65]:
train_X, test_X, train_Y, test_Y = train_test_split(filtered_left, filtered_right, test_size = 0.1)

In [66]:
train_X[:10], train_Y[:10]

(['pstaann',
  'bradey',
  'yakkun',
  'Tsukiatteiru',
  'botor',
  'diamarkaannn',
  'Brsepada',
  'Changanassery',
  'rightly',
  'dabudabu'],
 ['psta',
  'bradey',
  'yakkun',
  'tsukiatteiru',
  'botor',
  'amar',
  'sepada',
  'changanassery',
  'rightly',
  'dabudabu'])

In [67]:
with open('train_noisy_stem.json', 'w') as fopen:
    json.dump({'X': train_X, 'Y': train_Y}, fopen)

In [68]:
with open('test_noisy_stem.json', 'w') as fopen:
    json.dump({'X': test_X, 'Y': test_Y}, fopen)

In [69]:
len(train_X), len(test_X)

(1831677, 203520)

In [ ]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(filtered_left + filtered_right))

In [ ]:
import youtokentome as yttm

In [ ]:
stemmer_bpe = yttm.BPE.train(data='text.txt', vocab_size=1000, model='stemmer-v2.model',
              pad_id=0, unk_id=2, bos_id=3, eos_id=1)